# 보여주기 할려면 이거

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("C:/Users/SuperUser/Desktop/data/titanic_train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 여기서부터 보면 될듯,,? 간단한 전처리 및 학습 

col = ['Pclass','Sex','SibSp','Fare','Embarked']
y = y_train['Survived']
X_train = pd.get_dummies(X_train[col])
test = pd.get_dummies(X_test[col])


from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 7, n_estimators=200, random_state=2022)
model.fit(X_train,y)
pred = model.predict(test)
model.score(X_train,y)

0.8862359550561798

# 개열심히 분석할려면 이거

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("C:/Users/SuperUser/Desktop/data/titanic_train.csv")
x_train, x_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

In [62]:
x_train.info() # 보면 데이터 타입이 int형이랑 object 형 있는거 확인하고 null 값 확인하고

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 90 to 116
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Name         712 non-null    object 
 3   Sex          712 non-null    object 
 4   Age          575 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Cabin        170 non-null    object 
 10  Embarked     711 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB


In [63]:
x_train.isnull().sum() # null 값 확인 보면 AGE 랑 cabin, Embarked  null 값 있는거 확인

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          542
Embarked         1
dtype: int64

In [64]:
x_test.info() # 검증용 데이터도 동일한 과정을거친다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 210 to 45
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  179 non-null    int64  
 1   Pclass       179 non-null    int64  
 2   Name         179 non-null    object 
 3   Sex          179 non-null    object 
 4   Age          139 non-null    float64
 5   SibSp        179 non-null    int64  
 6   Parch        179 non-null    int64  
 7   Ticket       179 non-null    object 
 8   Fare         179 non-null    float64
 9   Cabin        34 non-null     object 
 10  Embarked     178 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 16.8+ KB


In [65]:
x_train.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          542
Embarked         1
dtype: int64

In [66]:
# 위에 확인해보면 Name 컬럼이랑 Id 값은 학습에 도움이 안되기 때문에 버려!

x_train.drop(columns=['PassengerId','Name'],inplace=True)
x_test.drop(columns=['PassengerId','Name'],inplace=True)

In [67]:
str_col = x_train.select_dtypes('object').columns
num_col = x_train.select_dtypes(exclude='object').columns
print(str_col)
print(num_col)

Index(['Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')


In [68]:
# null 값 처리하기
# 일단 수치형만 Null 값 처리해주면 될거야. 범주형은 one-hot encoding 해주거나 Labelencoder 해주면 알아서 
# 처리 해줄거야,,아마도

x_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
x_test['Age'].fillna(X_test['Age'].mean(),inplace=True)

In [69]:
# 정규화 시켜주기

from sklearn.preprocessing import StandardScaler,LabelEncoder


#수치형 데이터 정규화 (정규화 종류중 표준정규화 사용)
scaler = StandardScaler()
x_train[num_col] = scaler.fit_transform(x_train[num_col])
x_test[num_col] = scaler.fit_transform(x_test[num_col])


#범주형 데이터 정규화 시키기 (1)
# 근데 보통 이거 사용하는게 맘 편해
le = LabelEncoder()
for col in str_col:
    x_train[col] = le.fit_transform(x_train[col])
    x_test[col] = le.fit_transform(x_test[col])

    
# 범주형 데이터 정규화 시키기 (2)
# 얘는 범주형만 알아서 0,1 로 구분시켜주는 코드
# X_train = pd.get_dummies(X_train)
# X_test = pd.get_dummies(X_test)

In [81]:
# 이제 모델 학습 시키고 하면 됨

y = y_train['Survived'] # survived 예측해야 하니 이거만 호출
x_tr,x_te,y_tr,y_te = train_test_split(x_train,y,test_size=0.2,random_state=2022)

model_rf = RandomForestClassifier(random_state=2022)
model_rf.fit(x_tr,y_tr)
pred_rf = model_rf.predict(x_te)
proba_rf = model_rf.predict_proba(x_te)
score_rf = model_rf.score(x_te,y_te)
roc_auc_rf = roc_auc_score(y_te,proba_rf[:,1])

model_dt = DecisionTreeClassifier(random_state=2022)
model_dt.fit(x_tr,y_tr)
pred_dt = model_dt.predict(x_te)
proba_dt = model_dt.predict_proba(x_te)
score_dt = model_rf.score(x_te,y_te)
roc_auc_dt = roc_auc_score(y_te,proba_dt[:,1])

model_xg = XGBClassifier(random_state=2022)
model_xg.fit(x_tr,y_tr)
pred_xg = model_xg.predict(x_te)
proba_xg = model_xg.predict_proba(x_te)
score_xg = model_xg.score(x_te,y_te)
roc_auc_xg = roc_auc_score(y_te,proba_xg[:,1])


print('여기는 모델 자체 평가 지표')
print(f'RandomForestClassifier : {score_rf}')
print(f'DecisionTreeClassifier : {score_dt}')
print(f'XGBClassifier : {score_xg}')

print('-'*80)
print('여기는 roc_auc_score')
print(f'RandomForestClassifier roc_auc_score: {roc_auc_rf}')
print(f'DecisionTreeClassifier roc_auc_score: {roc_auc_dt}')
print(f'XGBClassifier roc_auc_score: {roc_auc_xg}')


repot1 = classification_report(y_te,pred_rf)
repot2 = classification_report(y_te,pred_dt)
repot4 = classification_report(y_te,pred_xg)
print('-'*80)
print('여기는 모델 혼동행렬표에 따른 각각의 평가지표')
print(f'RandomForestClassifier : \n{repot1}')
print('='*80)
print(f'DecisionTreeClassifier : \n{repot2}')
print('='*80)
print(f'XGBClassifier : \n{repot4}')

여기는 모델 자체 평가 지표
RandomForestClassifier : 0.8251748251748252
DecisionTreeClassifier : 0.8251748251748252
XGBClassifier : 0.8461538461538461
--------------------------------------------------------------------------------
여기는 roc_auc_score
RandomForestClassifier roc_auc_score: 0.8914256198347108
DecisionTreeClassifier roc_auc_score: 0.7056818181818182
XGBClassifier roc_auc_score: 0.8989669421487604
--------------------------------------------------------------------------------
여기는 모델 혼동행렬표에 따른 각각의 평가지표
RandomForestClassifier : 
              precision    recall  f1-score   support

           0       0.87      0.84      0.86        88
           1       0.76      0.80      0.78        55

    accuracy                           0.83       143
   macro avg       0.81      0.82      0.82       143
weighted avg       0.83      0.83      0.83       143

DecisionTreeClassifier : 
              precision    recall  f1-score   support

           0       0.78      0.74      0.76        88
     

# 추가 설명

이거 모델 ROC_AUC_SCORE는 그 뭐야,,ROC_Curve 그래프에서 면적 값을 의미하는데 1에 가까울수록 모델 성능은 좋은거,,추가 자료는 구글링 해봐 ㅋㅋ귀찮아,,,

혼동행렬표 
 - AccuracyScore- 정확도 : 전체 관측치 중 실제값과 예측치가 일치한 정도를 나타낸다. 정분류율은 범주의 분포가 균형을 이룰 때 효과적인 평가지표이다.
 - precision - 정밀도 : 모델의 예측값이 얼마나 정확하게 예측됐는가 나타내는 지표
 - recall - 재현율 : 실제 True 인 관측치 중 예측치가 적중한 정도 / 모형의 완정성 평가
 - f1-score : 정확도와 재현율의 조화평균. 정확도와 재현율에 같은 가중치를 부여하여 평균